In [1]:
# importing the tensorflow package
import tensorflow as tf

In [2]:
tf.test.is_built_with_cuda()

True

In [3]:
tf.config.list_physical_devices('GPU')

[]

# IMDB reviews (keras)

In [4]:
from __future__ import print_function

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.datasets import imdb

In [289]:
max_features = 30000

# обрезание текстов после данного количества слов (среди top max_features наиболее используемые слова)
maxlen = 150
batch_size = 50 # увеличьте значение для ускорения обучения

In [290]:
print('Загрузка данных...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

Загрузка данных...


In [291]:
x_train[0]

[1,
 14,
 22,
 16,
 43,
 530,
 973,
 1622,
 1385,
 65,
 458,
 4468,
 66,
 3941,
 4,
 173,
 36,
 256,
 5,
 25,
 100,
 43,
 838,
 112,
 50,
 670,
 22665,
 9,
 35,
 480,
 284,
 5,
 150,
 4,
 172,
 112,
 167,
 21631,
 336,
 385,
 39,
 4,
 172,
 4536,
 1111,
 17,
 546,
 38,
 13,
 447,
 4,
 192,
 50,
 16,
 6,
 147,
 2025,
 19,
 14,
 22,
 4,
 1920,
 4613,
 469,
 4,
 22,
 71,
 87,
 12,
 16,
 43,
 530,
 38,
 76,
 15,
 13,
 1247,
 4,
 22,
 17,
 515,
 17,
 12,
 16,
 626,
 18,
 19193,
 5,
 62,
 386,
 12,
 8,
 316,
 8,
 106,
 5,
 4,
 2223,
 5244,
 16,
 480,
 66,
 3785,
 33,
 4,
 130,
 12,
 16,
 38,
 619,
 5,
 25,
 124,
 51,
 36,
 135,
 48,
 25,
 1415,
 33,
 6,
 22,
 12,
 215,
 28,
 77,
 52,
 5,
 14,
 407,
 16,
 82,
 10311,
 8,
 4,
 107,
 117,
 5952,
 15,
 256,
 4,
 2,
 7,
 3766,
 5,
 723,
 36,
 71,
 43,
 530,
 476,
 26,
 400,
 317,
 46,
 7,
 4,
 12118,
 1029,
 13,
 104,
 88,
 4,
 381,
 15,
 297,
 98,
 32,
 2071,
 56,
 26,
 141,
 6,
 194,
 7486,
 18,
 4,
 226,
 22,
 21,
 134,
 476,
 26,
 480,
 5,
 1

In [292]:
# Retrieve the word index file mapping words to indices
word_index = imdb.get_word_index()
# Reverse the word index to obtain a dict mapping indices to words
inverted_word_index = dict((i, word) for (word, i) in word_index.items())
# Decode the first sequence in the dataset
decoded_sequence = " ".join(inverted_word_index[i] for i in x_train[0])

In [293]:
decoded_sequence

"the as you with out themselves powerful lets loves their becomes reaching had journalist of lot from anyone to have after out atmosphere never more room titillate it so heart shows to years of every never going villaronga help moments or of every chest visual movie except her was several of enough more with is now current film as you of mine potentially unfortunately of you than him that with out themselves her get for was camp of you movie sometimes movie that with scary but pratfalls to story wonderful that in seeing in character to of 70s musicians with heart had shadows they of here that with her serious to have does when from why what have critics they is you that isn't one will very to as itself with other tricky in of seen over landed for anyone of and br show's to whether from than out themselves history he name half some br of 'n odd was two most of mean for 1 any an boat she he should is thought frog but of script you not while history he heart to real at barrel but when fro

In [294]:
print(len(x_train), 'тренировочные последовательности')
print(len(x_test), 'тестовые последовательности')

25000 тренировочные последовательности
25000 тестовые последовательности


In [295]:
len(x_train[0])

218

In [296]:
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (25000, 150)
x_test shape: (25000, 150)


In [297]:
max_features

30000

In [298]:
layer = Embedding(max_features, 128)

In [299]:
layer(x_train[0]).shape

TensorShape([150, 128])

In [300]:
print('Построение модели...')
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

Построение модели...


In [301]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [302]:
print('Процесс обучения...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=1, # увеличьте при необходимости
          validation_data=(x_test, y_test))

Процесс обучения...
500/500 [==============================] - 138s 274ms/step - loss: 0.4033 - accuracy: 0.8161 - val_loss: 0.3528 - val_accuracy: 0.8554


In [303]:
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)

500/500 [==============================] - 17s 33ms/step - loss: 0.3528 - accuracy: 0.8554


In [304]:
print('Результат при тестировании:', score)
print('Тестовая точность:', acc)

Результат при тестировании: 0.35279402136802673
Тестовая точность: 0.855400025844574


## Анализ


- при увеличении max_features до 30000 и 50000 качество просело
- при увеличении maxlen до 120, 150 и 200 качество выросло, при этом значение 150 дало лучший результат; при 100 и 50 качество снизилось
- изменение LSTM показало результат хуже, чем до изменения, при этом менялось и количество нейронов, и dropout

Однако при повторных прогонах результаты отличались, и у меня не получилось добиться стабильного результат при изменении параметров. Например, max_features = 30000 иногда давал лучший результат, чем 20000, а maxlen = 150 давал рузельтат хуже, чем при 80.


- в итоге я поставил max_features = 30000 и maxlen = 150, что привело к улучшению точности до 0,8554 (при параметрах по умолчанию лучший результат был 0,8482)

Честно говоря, я в замешательстве от результатов, и не нашёл для себя чёткого объяснения разности результатов, кроме того, что система обучается на разных выборках.

# Генерация текста на основе книжки «Алиса в стране чудес»

In [308]:
import numpy as np
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.layers import SimpleRNN, LSTM, GRU
from tensorflow.keras.models import Sequential

In [309]:
# построчное чтение из примера с текстом 
with open("./alice_in_wonderland.txt", 'rb') as _in:
    lines = []
    for line in _in:
        line = line.strip().lower().decode("ascii", "ignore")
        if len(line) == 0:
            continue
        lines.append(line)
text = " ".join(lines)
chars = set(text)
nb_chars = len(chars)

In [310]:
len(chars)

55

In [311]:
# создание индекса символов и reverse mapping чтобы передвигаться между значениями numerical
# ID and a specific character. The numerical ID will correspond to a column
# ID и определенный символ. Numerical ID будет соответсвовать колонке
# число при использовании one-hot кодировки для представление входов символов
char2index = {c: i for i, c in enumerate(chars)}
index2char = {i: c for i, c in enumerate(chars)}

In [312]:
index2char

{0: '(',
 1: ' ',
 2: 'p',
 3: ')',
 4: '7',
 5: 't',
 6: '.',
 7: 'x',
 8: '[',
 9: '#',
 10: 'l',
 11: 'a',
 12: '%',
 13: '*',
 14: ',',
 15: '8',
 16: 's',
 17: '_',
 18: '2',
 19: '!',
 20: 'u',
 21: '5',
 22: 'f',
 23: '6',
 24: 'j',
 25: 'z',
 26: '3',
 27: 'h',
 28: '-',
 29: '?',
 30: 'g',
 31: 'c',
 32: '/',
 33: 'q',
 34: 'k',
 35: ']',
 36: '1',
 37: 'e',
 38: 'o',
 39: '@',
 40: 'm',
 41: 'v',
 42: 'd',
 43: 'i',
 44: 'y',
 45: 'w',
 46: 'r',
 47: '4',
 48: ':',
 49: 'n',
 50: '$',
 51: '0',
 52: ';',
 53: '9',
 54: 'b'}

In [331]:
# для удобства выберете фиксированную длину последовательность 10 символов 
SEQLEN, STEP = 15, 1
input_chars, label_chars = [], []

# конвертация data в серии разных SEQLEN-length субпоследовательностей
for i in range(0, len(text) - SEQLEN, STEP):
    input_chars.append(text[i: i + SEQLEN])
    label_chars.append(text[i + SEQLEN])

In [332]:
len(input_chars)

158768

In [333]:
input_chars[0], label_chars[0]

('project gutenbe', 'r')

In [334]:
input_chars[1], label_chars[1]

('roject gutenber', 'g')

In [335]:
# Вычисление one-hot encoding входных последовательностей X и следующего символа (the label) y

X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=np.bool)
y = np.zeros((len(input_chars), nb_chars), dtype=np.bool)
for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        X[i, j, char2index[ch]] = 1
    y[i, char2index[label_chars[i]]] = 1

<ipython-input-335-c8f27ba87c4d>:3: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=np.bool)
<ipython-input-335-c8f27ba87c4d>:4: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(input_chars), nb_chars), dtype=np.bool)


In [336]:
X.shape

(158768, 15, 55)

In [337]:
# установка ряда метапамертров  для нейронной сети и процесса тренировки
BATCH_SIZE, HIDDEN_SIZE = 128, 128
NUM_ITERATIONS = 10 # 25 должно быть достаточно
NUM_EPOCHS_PER_ITERATION = 1
NUM_PREDS_PER_EPOCH = 100

In [340]:
'''
Создание очень простой рекуррентной нейронной сети. В ней будет один реккурентный закодированный входной слой. 
За ним последует полносвязный слой связанный с набором возможных следующих символов, 
которые конвертированы в вероятностные результаты через стандартную softmax активацию 
с multi-class cross-encoding loss функцию ссылающуются на предсказание one-hot encoding лейбл символа
'''

model = Sequential()
model.add(
    LSTM(  # вы можете изменить эту часть на LSTM или SimpleRNN, чтобы попробовать альтернативы
        HIDDEN_SIZE,
        return_sequences=True,
        input_shape=(SEQLEN, nb_chars),
        unroll=True
    )
)

model.add(
    LSTM(  # вы можете изменить эту часть на LSTM или SimpleRNN, чтобы попробовать альтернативы
        HIDDEN_SIZE,
        return_sequences=False,
        unroll=True
    )
)

model.add(Dense(nb_chars))
model.add(Activation("softmax"))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

In [341]:
# выполнение серий тренировочных и демонстрационных итераций 
for iteration in range(NUM_ITERATIONS):

    # для каждой итерации запуск передачи данных в модель 
    print("=" * 50)
    print("Итерация #: %d" % (iteration))
    model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)

    # Select a random example input sequence.
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]

    # для числа шагов предсказаний использование текущей тренируемой модели 
    # конструирование one-hot encoding для тестирования input и добавление предсказания.
    print("Генерация из посева: %s" % (test_chars))
    print(test_chars, end="")
    for i in range(NUM_PREDS_PER_EPOCH):

        # здесь one-hot encoding.
        X_test = np.zeros((1, SEQLEN, nb_chars))
        for j, ch in enumerate(test_chars):
            X_test[0, j, char2index[ch]] = 1

        # осуществление предсказания с помощью текущей модели.
        pred = model.predict(X_test, verbose=0)[0]
        y_pred = index2char[np.argmax(pred)]

        # вывод предсказания добавленного к тестовому примеру 
        print(y_pred, end="")

        # инкрементация тестового примера содержащего предсказание
        test_chars = test_chars[1:] + y_pred

Итерация #: 0
1241/1241 [==============================] - 40s 30ms/step - loss: 2.4147
Генерация из посева: uld not help bu
uld not help bute and and the her her he the see he the her her he the see he the her her he the see he the her her==================================================
Итерация #: 1
1241/1241 [==============================] - 37s 30ms/step - loss: 1.9318
Генерация из посева:  you dont even 
 you dont even the doon the say hor had the sall the she doon the say hor had the sall the she doon the say hor had==================================================
Итерация #: 2
1241/1241 [==============================] - 41s 33ms/step - loss: 1.7193
Генерация из посева: s? the mouse di
s? the mouse dinter the more the more the more the more the more the more the more the more the more the more the m==================================================
Итерация #: 3
1241/1241 [==============================] - 41s 33ms/step - loss: 1.5796
Генерация из посева: the-way things 
th

## Анализ

- SEQLEN = 20, лучший текст:

Итерация #: 9
1241/1241 [==============================] - 44s 36ms/step - loss: 1.1613
Генерация из посева: the newspapers, at t
the newspapers, at the cat of her head in the cook as she had not a serpent, and the cat of her head in the cook as she 


- SEQLEN = 15, лучший текст:

Итерация #: 9
1241/1241 [==============================] - 34s 28ms/step - loss: 1.1591
Генерация из посева: s should be cle
s should be cleared at the table and distribution of the project gutenberg-tm electronic works in a little girls of


- После изменения GRU на LSTM, SEQLEN = 15, лучший текст:

Итерация #: 9
1241/1241 [==============================] - 41s 33ms/step - loss: 1.1869
Генерация из посева: tter. he came i
tter. he came in a long all the share was a long side of the soldiers of the same thing it was a long side of the s